In [ ]:
# Dependencies
import pandas as pd
import requests
# from config import api_key
from sqlalchemy import create_engine

## Scrape Current Movie Poster Links

Note: Since the following loop runs for near an hour, this scrape code is in markdown.
The dataset generated is saved to csv in Resources for working file.

In [ ]:
# Link to metadata csv file to collect imdb ID
# movies_metadata = "Resources/movies_metadata.csv"
# metadata_df = pd.read_csv(movies_metadata, encoding="UTF-8")

In [ ]:
# Collect dataset's movie IDs
# imdb_id_all = metadata_df['imdb_id']

In [ ]:
# Create lists for final dataset
# imdb_id = []
# poster_link = []

# Search source website for current movie poster links using IMDB ID
# for movie in imdb_id_all:
    
#     try:    
#         url = "https://api.themoviedb.org/3/find/" + movie + "?api_key=" + api_key + "&language=en-US&external_source=imdb_id"
#         movie_data = requests.get(url, stream=True).json()
        
#         poster = movie_data['movie_results'][0]['poster_path']
#         poster_link.append("https://image.tmdb.org/t/p/w600_and_h900_bestv2/" + poster)
#         imdb_id.append(movie)
        
#         print(f"The poster link for movie ID {movie} is saved.")
                
#     except:
#         print(f"No poster link available for {movie}.")
#         pass

In [ ]:
# Create dataframe to combine lists
# poster_links = pd.DataFrame(
#     {'imdb_id': imdb_id,
#      'poster_link': poster_link
#     })
# poster_links

In [ ]:
# Save dataframe as CSV
# poster_links.to_csv("Resources/poster_links.csv", index=False, header=True)

In [ ]:
# Pull csv file generated from above code for working data
poster_links_csv = "Resources/poster_links.csv"
poster_links = pd.read_csv(poster_links_csv, encoding="UTF-8")

## Extract & Transform Metadata

In [ ]:
# Store csv filepath in a variable
movies_metadata = "Resources/movies_metadata.csv"

In [ ]:
# Read our Data file with the pandas library
movies_metadata_df = pd.read_csv(movies_metadata, encoding="UTF-8")

In [ ]:
# Show just the header
movies_metadata_df.head()

In [ ]:
metadata = movies_metadata_df[['imdb_id', 
                               'title', 
                               'genres', 
                               'original_language',
                               'popularity', 
                               'production_countries', 
                               'release_date',
                               'runtime', 
                               'status', 
                               'vote_average', 
                               'vote_count']]

metadata = metadata.dropna(how='any')
released_films = pd.DataFrame(metadata.loc[(metadata['genres']!='[]') & (metadata['production_countries']!='[]'), :])
released_films

## Merge Metadata Sets

In [ ]:
metadata_complete = pd.merge(released_films, poster_links, on=['imdb_id'])
metadata_complete.head()

## Extract & Transform IMDB Title-Ratings TSV File

In [ ]:
# Link to source file and display head
ratings_file = "./Resources/title.ratings.tsv"
ratings_df = pd.read_csv(ratings_file, sep='\t')
ratings_df.head()

In [ ]:
# Rename the column headers
ratings_cols = ratings_df.rename(columns={"tconst": "imdb_id", 
                                          "averageRating": "avg_rating", 
                                          "numVotes": "num_votes"})

In [ ]:
# Clean the data by dropping duplicates and setting the index
ratings_cols.drop_duplicates("imdb_id", inplace=True)
ratings_cols.set_index("imdb_id", inplace=True)
ratings_cols.dropna(how='any')

## Creating Database Connection

In [ ]:
# create db connection
connection_string = "postgres:postgres@localhost:5432/cinophiles"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# confirm table names
engine.table_names()

## Load df into DB 

In [ ]:
metadata_complete.to_sql(name='TMDB', con=engine, if_exists='append', index=True)
ratings_cols.to_sql(name='IMDB_ratings', con=engine, if_exists='append', index=True)